# Lab 3: Intro to Using IBM GenAI Python Library

Welcome to the Lab 3. 

In the previous lab, we explored the challenges of prompt engineering; learning how to tweak our wording, choose different model plus optimize model parameters. Minor changes can significantly enhance the results generated by language models.

In this lab, we will apply our new knowledge to a real-world use case as we migrate from prompt engineering within the Watsonx.ai Workbench to coding prompts in Python. Using the [IBM GenAI Python library](https://ibm.github.io/ibm-generative-ai/) to programmatically interact with Watsonx.ai, we will use templates to streamline our interaction with the language model and maximize its potential.

The concept of Prompt Patterns provided by IBM's GenAI Python library allows you to construct templates that can be easily filled with specific information to generate a wide range of outputs. 

# Disclaimer
**The IBM GenAI Python library is currently in Beta and will change in the future.**

Also note that `ibm-generative-ai` requires `python>=3.9`.

## Recreating Prompt Builder Prompts Using GenAI Prompt Patterns

### Scenario: Personalized Recommendation for XYZ Retail Company <a id="step3"></a>

XYZ Retail is a popular online retail store that sells a wide range of products, including electronics, clothing, home goods, and more. They have a large customer base and want to provide a personalized shopping experience to enhance customer satisfaction and boost sales.

To achieve that goal, XYZ wants to leverage generative AI to create fact sheets about each of their customers. These fact sheets will summarize relevant information such as customer demographics (name, age, location), and purchase history. These fact sheets will help XYZ Retail's sales team build stronger customer relationships, increase customer satisfaction and drive repeat purchases.


You start by performing prompt engineering in Prompt Lab, and you might test base model output with an initial prompt like this:

![title](../images/prompt_with_example.png)

The model's recommendation is not accurate or useful as the customer Michael Jones had bought toys and games not outdoor activewear. Fortunately you learned in the Prompt Engineering lab that Few Shot Learning can help you obtain better results. 

What happens when we provide a few examples using Prompt Builder to guide the LLM into generating more meaningful recommendations. 

![title](../images/prompt_with_example.png?token=AAC77XO2FBIVGZZYWCD7BLLEUWKPC)


Great, the product recommendation for Michael Jones is much better.  However how do you productionize your few shot prompting to generate recommendations for all of XYZ Retail customers? Copy and pasting each customer's info into Prompt Builder would take too long.  

You'll need a programmatic solution.  Maybe you could even generate a large set of examples then use that for Tuning a model in Watsonx.ai.  But we're getting ahead of ourselves as you'll learn about building a Prompt Tuning dataset in a later lab.

Here is what you will learn in the following steps:

<p align="center">
  <img src="../images/scenario_flow_chart03.png?token=AAC77XKT7EJEOFXFCC5RXPTEUWKTQ" width="600"/>
</p>

## 1. Load the required libraries  <a id="step1"></a>

In [52]:
import os
from dotenv import load_dotenv
from genai.model import Credentials, Model
from genai.schemas import GenerateParams, ModelType
from genai.prompt_pattern import PromptPattern

## 2. Create a Factsheet for each customer using Prompt Patterns  <a id="step2"></a>

### **2.1 What are Prompt Patterns?**

The [PromptPattern class](https://ibm.github.io/ibm-generative-ai/rst_source/genai.prompt.prompt_pattern.html) in the [IBM GenAI Python library](https://ibm.github.io/ibm-generative-ai/) provides a flexible approach to creating prompts from structured templates.  We will use the PromptPattern class to simplify creation of our few shot prompts for XYZ Retail.

XYZ Retail has provided you their customer's data in .csv format. To generate prompts for each customer, you will need to transform the prompt that you engineered in Prompt Builder into a more useful programmatic format. Using the Prompts Pattern class, you can easily substitute customer data from a file to generate one or multiple prompts.

The PromptPattern class defines a schema where variables to replace are placed inside double curly braces "{{}}". These curly braces serve as a placeholder for the actual data that will be substituted into the template.

Let's see how this works in practice.

### **2.2 Creating a simple prompt from a template**

A prompt pattern can be created using the PromptPattern class from a string, file, or url. There are [additional PromptPattern examples](https://ibm.github.io/ibm-generative-ai/rst_source/examples/examples.html) provided in the IBM GenAI documentation

#### 2.2.1 Prompt Pattern From String

In [28]:
pattern = "input: {{name}} {{family_name}} is {{age}} and lives in {{location}}. They bought {{purchase_history}}"

prompt = PromptPattern.from_str(pattern)
prompt.sub("name", "Jane").sub("family_name", "Doe").sub("age", "43").sub("location", "San Francisco, CA").sub("purchase_history", "groceries, household goods and travel supplies")

prompt

input: Jane Doe is 43 and lives in San Francisco, CA. They bought groceries, household goods and travel supplies

#### 2.2.2 Prompt Pattern From File
Prompt patterns can also be stored as a yaml file:

In [39]:
_path_to_file = "../templates/customer_factsheet.yaml"

prompt = PromptPattern.from_file(_path_to_file)
print("TEMPLATE:\n" + str(prompt))

# Iterate over the numbers 1, 2 and 3 appending each to
for x in range(1, 4):
    prompt.sub(f"name_{x}", "Jane").sub(f"family_name_{x}", "Doe").sub(f"age_{x}", "43").sub(f"city_{x}", "San Francisco").sub(f"state_{x}", "CA")
    prompt.sub(f"purchase_history_{x}", "groceries, household goods and travel supplies")
    prompt.sub(f"recommendation_1_{x}", "Basket of organic fruits")
    prompt.sub(f"recommendation_2_{x}", "Lightweight carry-on suitcase")
print("\nPOPULATED TEMPLATE:\n" + str(prompt))

TEMPLATE:
input: "{{name_1}} {{family_name_1}} is {{age_1}} years old and lives in {{city_1}}, {{state_1}}. Their purchase history includes {{purchase_history_1}}."
output: "Recommendations:\n Item 1: {{recommendation_1_1}}\nItem 2: {{recommendation_2_1}}"
input: "{{name_2}} {{family_name_2}} is {{age_2}} years old and lives in {{city_2}}, {{state_2}}. Their purchase history includes {{purchase_history_2}}."
output: "Recommendations:\n Item 1: {{recommendation_1_2}}\nItem 2: {{recommendation_2_2}}"
input: "{{name_3}} {{family_name_3}} is {{age_3}} years old and lives in {{city_3}}, {{state_3}}. Their purchase history includes {{purchase_history_3}}."
output: ""


POPULATED TEMPLATE:
input: "Jane Doe is 43 years old and lives in San Francisco, CA. Their purchase history includes groceries, household goods and travel supplies."
output: "Recommendations:\n Item 1: Basket of organic fruits\nItem 2: Lightweight carry-on suitcase"
input: "Jane Doe is 43 years old and lives in San Francisco, 

## 3. Create Prompt Examples based on Customers Factsheet <a id="step3"></a>
The value of PromptPattern arises when generating a large number of prompts either as examples for bulk evaluation of an engineered prompt or for creation of a Tuning dataset

### 3.1 Bulk Creation of Prompts
We can now generate few shot Prompts from rows in a csv using "sub_all_from_csv". This could also be done from json.  Details can be found in the [PromptPattern class documentation](https://ibm.github.io/ibm-generative-ai/rst_source/genai.prompt.prompt_pattern.html)

In [67]:
_path_to_template_file = "../templates/customer_factsheet.yaml"
_path_to_csv_file = "../data/customer_factsheet.csv"

prompt = PromptPattern.from_file(_path_to_template_file)
print("TEMPLATE:\n" + str(prompt))

# We use a mapping table like below so each row in the CSV is mapped to the correct variable of the YAML template.
# The recommendations for rows 1 and 2 are used for few shot training while every row's 3rd recommendation
# is ignored as these are the recommendation for which we are evaluating our prompt's performance.
mapping = {
    "name": ["name_1", "name_2", "name_3"],
    "family_name": ["family_name_1", "family_name_2", "family_name_3"],
    "age": ["age_1", "age_2", "age_3"],
    "city": ["city_1", "city_2", "city_3"],
    "state": ["state_1", "state_2", "state_3"],
    "purchase_history": ["purchase_history_1", "purchase_history_2", "purchase_history_3"],
    "recommendation_1": ["recommendation_1_1", "recommendation_1_2", "recommendation_1_3"],
    "recommendation_2": ["recommendation_2_1", "recommendation_2_2", "recommendation_2_3"]
}

list_of_prompts = prompt.sub_all_from_csv(csv_path=_path_to_csv_file,col_to_var=mapping)

print("\nPOPULATED TEMPLATE:\n" + str(list_of_prompts[0]))

TEMPLATE:
input: "{{name_1}} {{family_name_1}} is {{age_1}} years old and lives in {{city_1}}, {{state_1}}. Their purchase history includes {{purchase_history_1}}."
output: "Recommendations:\n Item 1: {{recommendation_1_1}}\nItem 2: {{recommendation_2_1}}"
input: "{{name_2}} {{family_name_2}} is {{age_2}} years old and lives in {{city_2}}, {{state_2}}. Their purchase history includes {{purchase_history_2}}."
output: "Recommendations:\n Item 1: {{recommendation_1_2}}\nItem 2: {{recommendation_2_2}}"
input: "{{name_3}} {{family_name_3}} is {{age_3}} years old and lives in {{city_3}}, {{state_3}}. Their purchase history includes {{purchase_history_3}}."
output: ""


POPULATED TEMPLATE:
input: "John Smith is 30 years old and lives in San Francisco, CA. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It h

### 3.2 Additional Examples
You can explore [additional examples of using the Prompt Pattern](https://ibm.github.io/ibm-generative-ai/rst_source/examples/prompts.html)

## 4. Executing prompts <a id="step4"></a>
We can now execute these few shot prompts to see how well our engineered prompt works across numerous examples

### 4.1 Import Watsonx.ai access credentials and load model
Make sure you copied the .env file that you created earlier into the same directory as this notebook

In [86]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)
if api_key is None or api_endpoint is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = Credentials(api_key=api_key, api_endpoint=api_endpoint)

    params = GenerateParams(
        decoding_method="greedy",
        max_new_tokens=100,
        min_new_tokens=50,
        stream=False,
    )
    model = Model(ModelType.FLAN_UL2, params=params, credentials=creds)

### 4.2 Send prompts to Watsonx.ai

In [87]:
responses = model.generate_as_completed(list_of_prompts)
for i, response in enumerate(responses):
    lines = str(list_of_prompts[i]).strip().split("\n")
    user_description = str(lines[4])
    print(f"\n{user_description}")
    print(f"output: {response.generated_text}")


input: "Michael Jones is 40 years old and lives in Seattle, WA. Their purchase history includes Toys games sporting_goods."
output: Recommendations:n Item 1: X-Box One - The X-Box One is the latest gaming console from Microsoft. It has a high definition screen, and it is able to stream games from your Xbox 360 to the console. It also has an integrated Kinect sensor, which allows you to play games by simply moving your body.nItem 2: Xbox 360 - The Xbox 360 is a gaming console from Microsoft. It

input: "Ashley Brown is 20 years old and lives in Los Angeles, CA. Their purchase history includes Makeup skincare fashion."
output: Recommendations:n Item 1: Makeup - MAC Ruby Woo LipsticknItem 2: Skincare - Clinique Dramatically Different MoisturizernItem 3: Fashion - MAC x Patrick Starrr CollectionnItem 4: Fashion - MAC x Gwen Stefani CollectionnItem 5: Fashion - MAC x Gigi Hadid Collection

input: "Emily Johnson is 55 years old and lives in Dallas, TX. Their purchase history includes Furnit

### Few shot prompt analysis
These results are not bad.  An X-Box for a cusotmer with a history of buying toys and games.  Likewise cosmetics and furniture for the other two customers accurately reflects their purchase history.  

## 5. Congratulations
Congratulations on completing the lab and exploring the fascinating world of bulk creation of Few Shot Prompts using PromptPatterns! 

Through the practical use case of generating personalized product recommendations, you have witnessed the power of tailoring prompts to individual customer profiles. By incorporating customer-specific details and programmatically generating bulk examples, you can fine-tune the model for your specific use case, resulting in more accurate and tailored outputs. 

Continuously iterating and refining your prompts based on these examples will unlock the full potential of language models and enhance their performance across various domains. Keep experimenting and leveraging prompt engineering techniques to optimize your interactions with language models and drive impactful results in your projects.